In [10]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
df = pd.read_parquet("2024.parquet")
df_insert = df.iloc[:, [0, 3, 4, 5, 6, -1]] 
df_insert['codename'] = "股名"
df_insert['industry'] = "產業" 
df_insert.head(1)

,da,op,cl,lo,hi,ticker,codename,industry
0,113/01/02,"2,860.00","2,860.00","2,745.00","2,785.00",3008,股名,產業


In [11]:
df_insert['op'] = df['op'].apply(lambda x: float(x.replace(",", "").replace("-", "0")))
df_insert['hi'] = df['hi'].apply(lambda x: float(x.replace(",", "").replace("-", "0")))
df_insert['lo'] = df['lo'].apply(lambda x: float(x.replace(",", "").replace("-", "0")))
df_insert['cl'] = df['cl'].apply(lambda x: float(x.replace(",", "").replace("-", "0")))
df_insert['da'] = df['da'].apply(lambda x: f"{int(x[:3])+1911}-{x[4:6]}-{x[7:]}")
df_insert['da'] = pd.to_datetime(df_insert['da'])

In [12]:
import psycopg2

connection = psycopg2.connect(
    dbname="daily",
    user="mini",
    password="buddyrich134",
    host="localhost",
    port=5432
)

# Open a cursor to perform database operations
cursor = connection.cursor()


In [13]:
from tqdm import tqdm
for inedx, row in tqdm(df_insert.iloc[0:, :].iterrows()):
    insert_query = f'''
    INSERT INTO daily_price(da, codename, symbol, industry, op, hi, lo, cl)
    VALUES ('{row['da']}', '{row['codename']}', '{row['ticker']}', '{row['industry']}', '{row['op']}', '{row['hi']}', '{row['lo']}', '{row['cl']}');
    '''
    cursor.execute(insert_query)
    connection.commit()

126229it [00:21, 5864.20it/s]
